In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder.appName("PySpark Joins").getOrCreate()

# Create DataFrame 1
data1 = [("101", "John", "USA"),
         ("102", "Mike", "UK"),
         ("103", "Robert", "USA"),
         ("104", "James", "Canada")]

df1 = spark.createDataFrame(data1, ["emp_id", "name", "country"])

# Create DataFrame 2
data2 = [("101", "HR", 4000),
         ("102", "Finance", 5000),
         ("105", "Engineering", 6000),
         ("106", "Marketing", 3500)]

df2 = spark.createDataFrame(data2, ["emp_id", "department", "salary"])

df1.show()
df2.show()

+------+------+-------+
|emp_id|  name|country|
+------+------+-------+
|   101|  John|    USA|
|   102|  Mike|     UK|
|   103|Robert|    USA|
|   104| James| Canada|
+------+------+-------+

+------+-----------+------+
|emp_id| department|salary|
+------+-----------+------+
|   101|         HR|  4000|
|   102|    Finance|  5000|
|   105|Engineering|  6000|
|   106|  Marketing|  3500|
+------+-----------+------+



## Inner Join
* Inner Join: Matches rows in both DataFrames.

In [0]:
inner_join = df1.join(df2, df1.emp_id == df2.emp_id, "inner")
inner_join.show()


+------+----+-------+------+----------+------+
|emp_id|name|country|emp_id|department|salary|
+------+----+-------+------+----------+------+
|   101|John|    USA|   101|        HR|  4000|
|   102|Mike|     UK|   102|   Finance|  5000|
+------+----+-------+------+----------+------+



##Left Join
* Left Join: Keeps all rows from the left DataFrame and matches from the right.


In [0]:
inner_join = df1.join(df2, df1.emp_id == df2.emp_id, "left")
inner_join.show()


+------+------+-------+------+----------+------+
|emp_id|  name|country|emp_id|department|salary|
+------+------+-------+------+----------+------+
|   101|  John|    USA|   101|        HR|  4000|
|   102|  Mike|     UK|   102|   Finance|  5000|
|   103|Robert|    USA|  null|      null|  null|
|   104| James| Canada|  null|      null|  null|
+------+------+-------+------+----------+------+



##Right Join
* Right Join: Keeps all rows from the right DataFrame and matches from the left.


In [0]:
inner_join = df1.join(df2, df1.emp_id == df2.emp_id, "right")
inner_join.show()


+------+----+-------+------+-----------+------+
|emp_id|name|country|emp_id| department|salary|
+------+----+-------+------+-----------+------+
|   101|John|    USA|   101|         HR|  4000|
|   102|Mike|     UK|   102|    Finance|  5000|
|  null|null|   null|   105|Engineering|  6000|
|  null|null|   null|   106|  Marketing|  3500|
+------+----+-------+------+-----------+------+



## Full Outer Join
* Full Outer Join: Keeps all rows from both DataFrames, with NULL for missing matches.


In [0]:
inner_join = df1.join(df2, df1.emp_id == df2.emp_id, "outer")
inner_join.show()


+------+------+-------+------+-----------+------+
|emp_id|  name|country|emp_id| department|salary|
+------+------+-------+------+-----------+------+
|   101|  John|    USA|   101|         HR|  4000|
|   102|  Mike|     UK|   102|    Finance|  5000|
|   103|Robert|    USA|  null|       null|  null|
|   104| James| Canada|  null|       null|  null|
|  null|  null|   null|   105|Engineering|  6000|
|  null|  null|   null|   106|  Marketing|  3500|
+------+------+-------+------+-----------+------+



##Cross join
* Cross Join: Produces the Cartesian product.

In [0]:
cross_join = df1.crossJoin(df2)
cross_join.show()

+------+------+-------+------+-----------+------+
|emp_id|  name|country|emp_id| department|salary|
+------+------+-------+------+-----------+------+
|   101|  John|    USA|   101|         HR|  4000|
|   101|  John|    USA|   102|    Finance|  5000|
|   101|  John|    USA|   105|Engineering|  6000|
|   101|  John|    USA|   106|  Marketing|  3500|
|   102|  Mike|     UK|   101|         HR|  4000|
|   102|  Mike|     UK|   102|    Finance|  5000|
|   102|  Mike|     UK|   105|Engineering|  6000|
|   102|  Mike|     UK|   106|  Marketing|  3500|
|   103|Robert|    USA|   101|         HR|  4000|
|   103|Robert|    USA|   102|    Finance|  5000|
|   103|Robert|    USA|   105|Engineering|  6000|
|   103|Robert|    USA|   106|  Marketing|  3500|
|   104| James| Canada|   101|         HR|  4000|
|   104| James| Canada|   102|    Finance|  5000|
|   104| James| Canada|   105|Engineering|  6000|
|   104| James| Canada|   106|  Marketing|  3500|
+------+------+-------+------+-----------+------+


## Self Join
* Self Join: Joins a DataFrame to itself.


In [0]:
self_join = df1.alias("df1").join(df1.alias("df2"), col("df1.country") == col("df2.country"))
self_join.show()

+------+------+-------+------+------+-------+
|emp_id|  name|country|emp_id|  name|country|
+------+------+-------+------+------+-------+
|   104| James| Canada|   104| James| Canada|
|   102|  Mike|     UK|   102|  Mike|     UK|
|   101|  John|    USA|   101|  John|    USA|
|   101|  John|    USA|   103|Robert|    USA|
|   103|Robert|    USA|   101|  John|    USA|
|   103|Robert|    USA|   103|Robert|    USA|
+------+------+-------+------+------+-------+



##Anti Join
* Anti Join: Returns unmatched rows from the left DataFrame.


In [0]:
anti_join = df1.join(df2, df1.emp_id == df2.emp_id, "left_anti")
anti_join.show()

+------+------+-------+
|emp_id|  name|country|
+------+------+-------+
|   103|Robert|    USA|
|   104| James| Canada|
+------+------+-------+



##Semi join
* Semi Join: Returns matched rows from the left DataFrame, but not any columns from the right.


In [0]:
semi_join = df1.join(df2, df1.emp_id == df2.emp_id, "left_semi")
semi_join.show()

+------+----+-------+
|emp_id|name|country|
+------+----+-------+
|   101|John|    USA|
|   102|Mike|     UK|
+------+----+-------+

